# PC2_LP2_UNALM

# GRUPO: N°8

Integrantes:
- analisset7 - Ana Lisset Chucas Mera 20191296
- Huerta14   - Stefano Huerta Barba 20191305
- elian1300  - Elian Cuellar 20191300

- ¿Tiene API?
 No tiene API
 La página SI tiene API, sin embargo son de pago, por lo que no hemos requerido del uso de API y hemos hecho web scraping dinamico, es necesario mencionar que en nuestro trabajo usamos varios códigos de internet usados para el web scraping de otras páginas; https://www.udemy.com/course/curso-maestro-web-scraping/learn/lecture/18366012?start=180, https://es.stackoverflow.com/questions/414104/web-scrapping-en-sitios-con-carga-dinamica-js, https://gist.github.com/Nanguage/f35b1f66a968c204dc13dc035a163d7e, https://selenium-python.readthedocs.io/installation.html#drivers, https://selenium-python.readthedocs.io/navigating.html , https://www.w3schools.com/python/python_try_except.aspy muchas otras.

NUESTRO TRABAJO SE ENCUENTRA EN EL REPOSITORIO COLABORATIVO:
- https://github.com/analisset7/PC2_LP2_UNALM

In [10]:
import pandas as pd 
data= pd.read_excel('CHGenesOrdenadosConVecinos.xlsx',engine='openpyxl',skiprows=[0],usecols=range(8)) # Leemos el archivo de excel
data.head() #mostramos la lectura

,Gen Abrev,Category,Name of the gene,Inicio,Final,RNASeq,Microarray,SAGE
0,TRNAS-GGA,NaN,NaN,32091689,32091761,NaN,NaN,NaN
1,LOC102173449,NaN,NaN,32837131,32838076,NaN,NaN,NaN
2,LOC102169333,NaN,NaN,33089112,33090132,NaN,NaN,NaN
3,LOC108636757,NaN,NaN,33764252,33768221,NaN,NaN,NaN
4,LOC102187712,NaN,NaN,33810174,33811741,NaN,NaN,NaN


In [17]:
Gen=data["Gen Abrev"] # Extraemos la primera columna y la guardamos en "Gen"
Gen=Gen.tolist()      # Lo tranformamos en una lista
Gen[0:5]  

['TRNAS-GGA', 'LOC102173449', 'LOC102169333', 'LOC108636757', 'LOC102187712']

In [18]:
# en la página web vemos que cada GEN tiene un link en el cúal se encuentran las diferentes caracteristicas 
# que tenemos que extraer al excel, por lo que encontramos un patron en los links, los cuales los vamos a almacenar 
# en una lista llamada "urls" y con el párametro "gene" ubicado en el HTML vamos a hacer un for en el
# "+name" dentro del URL semilla.

urls=[]              # Creamos una lista vacia llamada urls
for name in Gen:         # Hacemos un bucle for para cada elemento de nuestra lista Gen
    url = 'https://www.genecards.org/cgi-bin/carddisp.pl?gene='+name+'#expression' # Almacenamos el url para cada gen "+name"
    #print(url)
    urls.append(url) # Agregamos cada url a la lista links

Esta página web que estamos scrapeando es una página que requiere del uso de Selenium y la instalación de webdriver.Chrome, para lo cual adjuntamos los 2 links de ayuda para su correcta instalación.
https://selenium-python.readthedocs.io/navigating.html , https://www.youtube.com/watch?v=qYqGGrAA_IA&t=3s

In [19]:
# LIBRERIAS
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import re 
from selenium.webdriver.common.action_chains import ActionChains
from time import sleep

In [27]:
# hay información que nos pide el excel que se encuentra en la página como imagenes, por eso:
# Abrimos el "web driver" con todos los URLs de cada uno de los genes.
# puede demorar largo tiempo ya que abre link por link de todos los genes
link='https://www.genecards.org/cgi-bin/carddisp.pl?gene=HTR1F#expression'
driver = webdriver.Chrome(executable_path="C:/Users/Ana/Desktop/PC2_LP2_UNALM/chromedriver.exe") # Simulación chrome
lista_html=[]              
for link in urls:   
    driver.get(link) 
   
    try:
        element = driver.find_element_by_id("expressionImage") 
        soup = BeautifulSoup(driver.page_source, features='html.parser')
        while(soup.find("area")==None):
            driver.get(link) 
            soup = BeautifulSoup(driver.page_source, features='html.parser')
        lista_html.append(soup)
    except:
        lista_html.append(None)
        continue
driver.quit()  # cerrar navegador

In [30]:
genes=[]
for gen in lista_html:
    G=[]
    if gen == None:
        G.append(None)
    else:
        soup=gen
        G.append(soup.find("span",{"class":"gc-category"}).text)
        G.append(soup.find_all("span",{"class":"aliasMainName"})[1].text)
        lista =soup.find_all("area",{"title":re.compile('.*Breast.*')})
        G.append(lista[0]["title"].split(",")[0]+', '+lista[3]["title"].split(",")[0])
        G.append(lista[1]["title"].split(",")[0])
        G.append(lista[2]["title"].split(",")[0])
    genes.append(G)
print(genes)

[[None], [None], [None], [None], [None], ['(Protein Coding)', 'Vestigial Like Family Member 3', '25, 44', 'Breast - No Data', '0'], ['(Protein Coding)', 'Charged Multivesicular Body Protein 2B', '42, 44', 'Breast - No Data', '21'], ['(Protein Coding)', 'POU Class 1 Homeobox 1', '2, 0', 'Breast - No Data', '3'], ['(Protein Coding)', '5-Hydroxytryptamine Receptor 1F', '6, 3', 'Breast - No Data', 'Breast - No Data'], [None], ['(Protein Coding)', 'CGG Triplet Repeat Binding Protein 1', '43, 49', 'Breast - No Data', '18'], ['(Protein Coding)', 'Zinc Finger Protein 654', '23, 20', 'Breast - No Data', '0'], [None], [None], ['(Protein Coding)', 'Solute Carrier Family 12 Member 8', '10, 11', 'Breast - No Data', 'Breast - No Data'], [None], ['(Protein Coding)', 'Zinc Finger Protein 148', '22, 26', 'Breast - No Data', '3'], ['(Protein Coding)', 'Sorting Nexin 4', '35, 38', 'Breast - No Data', '6'], ['(Protein Coding)', 'Oxysterol Binding Protein Like 11', '22, 27', 'Breast - No Data', 'Breast - N

In [32]:
import numpy as np
import pandas as pd
#es importante que nuestro archivo sea un data frame, para su facil manejo
GENES=pd.DataFrame(genes, columns=['Category','Name of the gene','RNASeq','Microarray','SAGE'])
#guardamos las columnas:
colum1=data['Gen Abrev'] # Guardamos la columna Gen Abrev en el objeto llamado t1
colum2=GENES.iloc[:,[0,1]]
colum3=data[['Inicio','Final']] # Guardamos las columnas Inicio y Final en el objeto llamado t3
colum4=GENES.iloc[:,2:5]
A = pd.concat([colum1,colum2,colum3,colum4], axis=1)
A = A.fillna(value='NaN')
A.head(10)

,Gen Abrev,Category,Name of the gene,Inicio,Final,RNASeq,Microarray,SAGE
0,TRNAS-GGA,NaN,NaN,32091689,32091761,NaN,NaN,NaN
1,LOC102173449,NaN,NaN,32837131,32838076,NaN,NaN,NaN
2,LOC102169333,NaN,NaN,33089112,33090132,NaN,NaN,NaN
3,LOC108636757,NaN,NaN,33764252,33768221,NaN,NaN,NaN
4,LOC102187712,NaN,NaN,33810174,33811741,NaN,NaN,NaN
5,VGLL3,(Protein Coding),Vestigial Like Family Member 3,33882149,33934641,"25, 44",Breast - No Data,0
6,CHMP2B,(Protein Coding),Charged Multivesicular Body Protein 2B,34199383,34231630,"42, 44",Breast - No Data,21
7,POU1F1,(Protein Coding),POU Class 1 Homeobox 1,34235896,34251973,"2, 0",Breast - No Data,3
8,HTR1F,(Protein Coding),5-Hydroxytryptamine Receptor 1F,34811253,34812505,"6, 3",Breast - No Data,Breast - No Data
9,LOC102188976,NaN,NaN,34826096,34855603,NaN,NaN,NaN


In [35]:
A.to_excel("CHGenesOrdenadosConVecinos1.xlsx",index=False)